In [1]:
# adding correlation coefficients for raw signals 

In [2]:
import numpy as np

from sklearn import preprocessing, svm
from sklearn.metrics import confusion_matrix, classification_report, f1_score
import matplotlib.pyplot as plt

import scipy.stats as st
from scipy.fftpack import fft, fftfreq
from scipy.signal import argrelextrema
import operator

In [3]:
# load body acceleration raw signals 
# x axis
bx_train = np.loadtxt('./HARDataset/train/Inertial Signals/body_acc_x_train.txt')
bx_test = np.loadtxt('./HARDataset/test/Inertial Signals/body_acc_x_test.txt')
# y axis
by_train = np.loadtxt('./HARDataset/train/Inertial Signals/body_acc_y_train.txt')
by_test = np.loadtxt('./HARDataset/test/Inertial Signals/body_acc_y_test.txt')
# z axis
bz_train = np.loadtxt('./HARDataset/train/Inertial Signals/body_acc_z_train.txt')
bz_test = np.loadtxt('./HARDataset/test/Inertial Signals/body_acc_z_test.txt')

In [4]:
# load total acceleration raw signals 
# x axis
tx_train = np.loadtxt('./HARDataset/train/Inertial Signals/total_acc_x_train.txt')
tx_test = np.loadtxt('./HARDataset/test/Inertial Signals/total_acc_x_test.txt')
# y axis
ty_train = np.loadtxt('./HARDataset/train/Inertial Signals/total_acc_y_train.txt')
ty_test = np.loadtxt('./HARDataset/test/Inertial Signals/total_acc_y_test.txt')
# z axis
tz_train = np.loadtxt('./HARDataset/train/Inertial Signals/total_acc_z_train.txt')
tz_test = np.loadtxt('./HARDataset/test/Inertial Signals/total_acc_z_test.txt')

In [5]:
# load body gyroscope raw signals 
# x axis
gx_train = np.loadtxt('./HARDataset/train/Inertial Signals/body_gyro_x_train.txt')
gx_test = np.loadtxt('./HARDataset/test/Inertial Signals/body_gyro_x_test.txt')
# y axis
gy_train = np.loadtxt('./HARDataset/train/Inertial Signals/body_gyro_y_train.txt')
gy_test = np.loadtxt('./HARDataset/test/Inertial Signals/body_gyro_y_test.txt')
# z axis
gz_train = np.loadtxt('./HARDataset/train/Inertial Signals/body_gyro_z_train.txt')
gz_test = np.loadtxt('./HARDataset/test/Inertial Signals/body_gyro_z_test.txt')

In [6]:
bx_train.shape, bx_test.shape

((7352, 128), (2947, 128))

In [7]:
tx_train.shape, tx_test.shape

((7352, 128), (2947, 128))

In [8]:
gx_train.shape, gx_test.shape

((7352, 128), (2947, 128))

In [9]:
# load label vectors
y_train = np.loadtxt('./HARDataset/train/y_train.txt')
y_test = np.loadtxt('./HARDataset/test/y_test.txt')

In [10]:
y_train.shape, y_test.shape

((7352,), (2947,))

In [11]:
label_names = ['Walking', 'Walking upstairs', 'Walking downstairs', 'Sitting', 'Standing', 'Laying']

In [12]:
def stat_area_features(x, Te=1.0):
    # mean
    mean_ts = np.mean(x, axis=1).reshape(-1, 1)
    # max
    max_ts = np.amax(x, axis=1).reshape(-1, 1)
    # min
    min_ts = np.amin(x, axis=1).reshape(-1, 1)
    # std
    std_ts = np.std(x, axis=1).reshape(-1, 1)
    # skew
    skew_ts = st.skew(x, axis=1).reshape(-1, 1)
    # kurtosis
    kurtosis_ts = st.kurtosis(x, axis=1).reshape(-1, 1)
    # interquartile range
    iqr_ts = st.iqr(x, axis=1).reshape(-1, 1)
    # median absolute deviation
    mad_ts = np.median(np.sort(abs(x - np.mean(x, axis=1).reshape(-1, 1)), axis=1), axis=1).reshape(-1, 1)
    # area under curve
    area_ts = np.trapz(x, axis=1, dx=Te).reshape(-1, 1)
    # area under curve ** 2
    sq_area_ts = np.trapz(x ** 2, axis=1, dx=Te).reshape(-1, 1)
    
    return np.concatenate((mean_ts, max_ts, min_ts, std_ts, skew_ts, kurtosis_ts, iqr_ts, 
                           mad_ts, area_ts, sq_area_ts), axis=1)

In [13]:
# stats for train data

In [14]:
bx_train_stats = stat_area_features(bx_train)
by_train_stats = stat_area_features(by_train)
bz_train_stats = stat_area_features(bz_train)

In [15]:
bx_train_stats.shape, by_train_stats.shape, bz_train_stats.shape

((7352, 10), (7352, 10), (7352, 10))

In [16]:
tx_train_stats = stat_area_features(tx_train)
ty_train_stats = stat_area_features(ty_train)
tz_train_stats = stat_area_features(tz_train)

In [17]:
tx_train_stats.shape, ty_train_stats.shape, tz_train_stats.shape

((7352, 10), (7352, 10), (7352, 10))

In [18]:
gx_train_stats = stat_area_features(gx_train)
gy_train_stats = stat_area_features(gy_train)
gz_train_stats = stat_area_features(gz_train)

In [19]:
gx_train_stats.shape, gy_train_stats.shape, gz_train_stats.shape

((7352, 10), (7352, 10), (7352, 10))

In [20]:
# stats for test data

In [21]:
bx_test_stats = stat_area_features(bx_test)
by_test_stats = stat_area_features(by_test)
bz_test_stats = stat_area_features(bz_test)

In [22]:
bx_test_stats.shape, by_test_stats.shape, bz_test_stats.shape

((2947, 10), (2947, 10), (2947, 10))

In [23]:
tx_test_stats = stat_area_features(tx_test)
ty_test_stats = stat_area_features(ty_test)
tz_test_stats = stat_area_features(tz_test)

In [24]:
tx_test_stats.shape, ty_test_stats.shape, tz_test_stats.shape

((2947, 10), (2947, 10), (2947, 10))

In [25]:
gx_test_stats = stat_area_features(gx_test)
gy_test_stats = stat_area_features(gy_test)
gz_test_stats = stat_area_features(gz_test)

In [26]:
gx_test_stats.shape, gy_test_stats.shape, gz_test_stats.shape

((2947, 10), (2947, 10), (2947, 10))

In [27]:
# jerk for train data

In [28]:
bx_train_jerk = stat_area_features((bx_train[:, 1:] - bx_train[:, :-1])/1.0)
by_train_jerk = stat_area_features((by_train[:, 1:] - by_train[:, :-1])/1.0)
bz_train_jerk = stat_area_features((bz_train[:, 1:] - bz_train[:, :-1])/1.0)

In [29]:
bx_train_jerk.shape, by_train_jerk.shape, bz_train_jerk.shape

((7352, 10), (7352, 10), (7352, 10))

In [30]:
tx_train_jerk = stat_area_features((tx_train[:, 1:] - tx_train[:, :-1])/1.0)
ty_train_jerk = stat_area_features((ty_train[:, 1:] - ty_train[:, :-1])/1.0)
tz_train_jerk = stat_area_features((tz_train[:, 1:] - tz_train[:, :-1])/1.0)

In [31]:
tx_train_jerk.shape, ty_train_jerk.shape, tz_train_jerk.shape

((7352, 10), (7352, 10), (7352, 10))

In [32]:
gx_train_jerk = stat_area_features((gx_train[:, 1:] - gx_train[:, :-1])/1.0)
gy_train_jerk = stat_area_features((gy_train[:, 1:] - gy_train[:, :-1])/1.0)
gz_train_jerk = stat_area_features((gz_train[:, 1:] - gz_train[:, :-1])/1.0)

In [33]:
gx_train_jerk.shape, gy_train_jerk.shape, gz_train_jerk.shape

((7352, 10), (7352, 10), (7352, 10))

In [34]:
# jerk for test data

In [35]:
bx_test_jerk = stat_area_features((bx_test[:, 1:] - bx_test[:, :-1])/1.0)
by_test_jerk = stat_area_features((by_test[:, 1:] - by_test[:, :-1])/1.0)
bz_test_jerk = stat_area_features((bz_test[:, 1:] - bz_test[:, :-1])/1.0)

In [36]:
bx_test_jerk.shape, by_test_jerk.shape, bz_test_jerk.shape

((2947, 10), (2947, 10), (2947, 10))

In [37]:
tx_test_jerk = stat_area_features((tx_test[:, 1:] - tx_test[:, :-1])/1.0)
ty_test_jerk = stat_area_features((ty_test[:, 1:] - ty_test[:, :-1])/1.0)
tz_test_jerk = stat_area_features((tz_test[:, 1:] - tz_test[:, :-1])/1.0)

In [38]:
tx_test_jerk.shape, ty_test_jerk.shape, tz_test_jerk.shape

((2947, 10), (2947, 10), (2947, 10))

In [39]:
gx_test_jerk = stat_area_features((gx_test[:, 1:] - gx_test[:, :-1])/1.0)
gy_test_jerk = stat_area_features((gy_test[:, 1:] - gy_test[:, :-1])/1.0)
gz_test_jerk = stat_area_features((gz_test[:, 1:] - gz_test[:, :-1])/1.0)

In [40]:
gx_test_jerk.shape, gy_test_jerk.shape, gz_test_jerk.shape

((2947, 10), (2947, 10), (2947, 10))

In [41]:
def frequency_domain_features(x, Te=1.0):
    
    # as DFT coefficients and their corresponding frequencies are symetrical arrays with respect to
    # the middle of the array, need to control for whether samples in x are odd or even to then split arrays
    if x.shape[1]%2 == 0:
        N = int(x.shape[1]/2)
    else: 
        N = int(x.shape[1]/2) - 1
    xf = np.repeat(fftfreq(x.shape[1], d=1.0)[:N].reshape(1, -1), x.shape[0], axis=0) # frequencies
    dft = np.abs(fft(x, axis=1))[:, :N] # DFT coefficients
    
    # stat and area features
    dft_features = stat_area_features(dft)
    # weighted mean freq
    dft_weighted_mean_f = np.average(xf, axis=1, weights=dft).reshape(-1, 1)
    # first 5 DFT coefficients
    dft_first_coeff = dft[:, :5]
    # first 5 local maxima of DFT coefficients and their corresponding frequencies
    dft_max_coeff = np.zeros((x.shape[0], 5))
    dft_max_coeff_f = np.zeros((x.shape[0], 5))
    for row in range(x.shape[0]):
        # find indexes of all local maximas
        extrema_indiv = argrelextrema(dft[row, :], np.greater, axis=0)
        # make list of tuples (DFT_i, f_i) of all local maxima
        extrema_row = sorted([(dft[row, :][j], xf[row, j]) for j in extrema_indiv[0]],
                             key=operator.itemgetter(0), reverse=True)[:5]
        for i, ext in enumerate(extrema_row):
            dft_max_coeff[row, i] = ext[0]
            dft_max_coeff_f[row, i] = ext[1]
    
    return np.concatenate((dft_features, dft_weighted_mean_f, dft_first_coeff, 
                           dft_max_coeff, dft_max_coeff_f), axis=1)

In [42]:
# dft for train data

In [43]:
bx_train_dft = frequency_domain_features(bx_train)
by_train_dft = frequency_domain_features(by_train)
bz_train_dft = frequency_domain_features(bz_train)

In [44]:
bx_train_dft.shape, by_train_dft.shape, bz_train_dft.shape

((7352, 26), (7352, 26), (7352, 26))

In [45]:
tx_train_dft = frequency_domain_features(tx_train)
ty_train_dft = frequency_domain_features(ty_train)
tz_train_dft = frequency_domain_features(tz_train)

In [46]:
tx_train_dft.shape, ty_train_dft.shape, tz_train_dft.shape

((7352, 26), (7352, 26), (7352, 26))

In [47]:
gx_train_dft = frequency_domain_features(gx_train)
gy_train_dft = frequency_domain_features(gy_train)
gz_train_dft = frequency_domain_features(gz_train)

In [48]:
gx_train_dft.shape, gy_train_dft.shape, gz_train_dft.shape

((7352, 26), (7352, 26), (7352, 26))

In [49]:
# dft for test data

In [50]:
bx_test_dft = frequency_domain_features(bx_test)
by_test_dft = frequency_domain_features(by_test)
bz_test_dft = frequency_domain_features(bz_test)

In [51]:
bx_test_dft.shape, by_test_dft.shape, bz_test_dft.shape

((2947, 26), (2947, 26), (2947, 26))

In [52]:
tx_test_dft = frequency_domain_features(tx_test)
ty_test_dft = frequency_domain_features(ty_test)
tz_test_dft = frequency_domain_features(tz_test)

In [53]:
tx_test_dft.shape, ty_test_dft.shape, tz_test_dft.shape

((2947, 26), (2947, 26), (2947, 26))

In [54]:
gx_test_dft = frequency_domain_features(gx_test)
gy_test_dft = frequency_domain_features(gy_test)
gz_test_dft = frequency_domain_features(gz_test)

In [55]:
gx_test_dft.shape, gy_test_dft.shape, gz_test_dft.shape

((2947, 26), (2947, 26), (2947, 26))

In [56]:
# dft_jerk for train data

In [57]:
bx_train_dft_jerk = frequency_domain_features((bx_train[:, 1:] - bx_train[:, :-1])/1.0)
by_train_dft_jerk = frequency_domain_features((by_train[:, 1:] - by_train[:, :-1])/1.0)
bz_train_dft_jerk = frequency_domain_features((bz_train[:, 1:] - bz_train[:, :-1])/1.0)

In [58]:
bx_train_dft_jerk.shape, by_train_dft_jerk.shape, bz_train_dft_jerk.shape

((7352, 26), (7352, 26), (7352, 26))

In [59]:
tx_train_dft_jerk = frequency_domain_features((tx_train[:, 1:] - tx_train[:, :-1])/1.0)
ty_train_dft_jerk = frequency_domain_features((ty_train[:, 1:] - ty_train[:, :-1])/1.0)
tz_train_dft_jerk = frequency_domain_features((tz_train[:, 1:] - tz_train[:, :-1])/1.0)

In [60]:
tx_train_dft_jerk.shape, ty_train_dft_jerk.shape, tz_train_dft_jerk.shape

((7352, 26), (7352, 26), (7352, 26))

In [61]:
gx_train_dft_jerk = frequency_domain_features((gx_train[:, 1:] - gx_train[:, :-1])/1.0)
gy_train_dft_jerk = frequency_domain_features((gy_train[:, 1:] - gy_train[:, :-1])/1.0)
gz_train_dft_jerk = frequency_domain_features((gz_train[:, 1:] - gz_train[:, :-1])/1.0)

In [62]:
gx_train_dft_jerk.shape, gy_train_dft_jerk.shape, gz_train_dft_jerk.shape

((7352, 26), (7352, 26), (7352, 26))

In [63]:
# dft_jerk for test data

In [64]:
bx_test_dft_jerk = frequency_domain_features((bx_test[:, 1:] - bx_test[:, :-1])/1.0)
by_test_dft_jerk = frequency_domain_features((by_test[:, 1:] - by_test[:, :-1])/1.0)
bz_test_dft_jerk = frequency_domain_features((bz_test[:, 1:] - bz_test[:, :-1])/1.0)

In [65]:
bx_test_dft_jerk.shape, by_test_dft_jerk.shape, bz_test_dft_jerk.shape

((2947, 26), (2947, 26), (2947, 26))

In [66]:
tx_test_dft_jerk = frequency_domain_features((tx_test[:, 1:] - tx_test[:, :-1])/1.0)
ty_test_dft_jerk = frequency_domain_features((ty_test[:, 1:] - ty_test[:, :-1])/1.0)
tz_test_dft_jerk = frequency_domain_features((tz_test[:, 1:] - tz_test[:, :-1])/1.0)

In [67]:
tx_test_dft_jerk.shape, ty_test_dft_jerk.shape, tz_test_dft_jerk.shape

((2947, 26), (2947, 26), (2947, 26))

In [68]:
gx_test_dft_jerk = frequency_domain_features((gx_test[:, 1:] - gx_test[:, :-1])/1.0)
gy_test_dft_jerk = frequency_domain_features((gy_test[:, 1:] - gy_test[:, :-1])/1.0)
gz_test_dft_jerk = frequency_domain_features((gz_test[:, 1:] - gz_test[:, :-1])/1.0)

In [69]:
gx_test_dft_jerk.shape, gy_test_dft_jerk.shape, gz_test_dft_jerk.shape

((2947, 26), (2947, 26), (2947, 26))

In [70]:
# Correlation coefficients for raw signals between axis

In [73]:
def corr_coeffs(x, y, z):
    corr = np.empty((x.shape[0], 3))
    for row in range(x.shape[0]):
        xyz_matrix = np.concatenate((x[row, :].reshape(1, -1), y[row, :].reshape(1, -1), 
                                     z[row, :].reshape(1, -1)), axis=0)
        corr[row, 0] = np.corrcoef(xyz_matrix)[0, 1]
        corr[row, 1] = np.corrcoef(xyz_matrix)[0, 2]
        corr[row, 2] = np.corrcoef(xyz_matrix)[1, 2]
        
    return corr 

In [77]:
corr_b_train = corr_coeffs(bx_train, by_train, bz_train)
corr_t_train = corr_coeffs(tx_train, ty_train, tz_train)
corr_g_train = corr_coeffs(gx_train, gy_train, gz_train)

In [78]:
corr_b_train.shape, corr_t_train.shape, corr_g_train.shape

((7352, 3), (7352, 3), (7352, 3))

In [79]:
corr_b_test = corr_coeffs(bx_test, by_test, bz_test)
corr_t_test = corr_coeffs(tx_test, ty_test, tz_test)
corr_g_test = corr_coeffs(gx_test, gy_test, gz_test)

In [80]:
corr_b_test.shape, corr_t_test.shape, corr_g_test.shape

((2947, 3), (2947, 3), (2947, 3))

In [28]:
# creating X_train and X_test 

In [82]:
X_train = np.concatenate((bx_train_stats, by_train_stats, bz_train_stats, 
                          tx_train_stats, ty_train_stats, tz_train_stats, 
                          gx_train_stats, gy_train_stats, gz_train_stats, 
                          bx_train_jerk, by_train_jerk, bz_train_jerk, 
                          tx_train_jerk, ty_train_jerk, tz_train_jerk, 
                          gx_train_jerk, gy_train_jerk, gz_train_jerk,
                          bx_train_dft, by_train_dft, bz_train_dft, 
                          tx_train_dft, ty_train_dft, tz_train_dft, 
                          gx_train_dft, gy_train_dft, gz_train_dft, 
                          bx_train_dft_jerk, by_train_dft_jerk, bz_train_dft_jerk, 
                          tx_train_dft_jerk, ty_train_dft_jerk, tz_train_dft_jerk, 
                          gx_train_dft_jerk, gy_train_dft_jerk, gz_train_dft_jerk, 
                          corr_b_train, corr_t_train, corr_g_train), axis=1)

In [83]:
X_test = np.concatenate((bx_test_stats, by_test_stats, bz_test_stats, 
                          tx_test_stats, ty_test_stats, tz_test_stats, 
                          gx_test_stats, gy_test_stats, gz_test_stats, 
                          bx_test_jerk, by_test_jerk, bz_test_jerk, 
                          tx_test_jerk, ty_test_jerk, tz_test_jerk, 
                          gx_test_jerk, gy_test_jerk, gz_test_jerk,
                          bx_test_dft, by_test_dft, bz_test_dft, 
                          tx_test_dft, ty_test_dft, tz_test_dft, 
                          gx_test_dft, gy_test_dft, gz_test_dft, 
                          bx_test_dft_jerk, by_test_dft_jerk, bz_test_dft_jerk, 
                          tx_test_dft_jerk, ty_test_dft_jerk, tz_test_dft_jerk, 
                          gx_test_dft_jerk, gy_test_dft_jerk, gz_test_dft_jerk, 
                          corr_b_test, corr_t_test, corr_g_test), axis=1)

In [85]:
X_train.shape, X_test.shape 

((7352, 657), (2947, 657))

In [86]:
# scaling data makes big differnce! to all! 

In [87]:
from sklearn.preprocessing import StandardScaler

In [88]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [89]:
#can one do a simple log reg first?
from sklearn.linear_model import LogisticRegression

In [90]:
logreg = LogisticRegression()

In [91]:
logreg.fit(X_train, y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [93]:
logreg.score(X_train, y_train) # adding corr: 0.99 to 0.99

0.9917029379760609

In [94]:
y_pred_logreg = logreg.predict(X_test)

In [96]:
print(classification_report(y_test, y_pred_logreg)) # adding corr accuracy: 0.95 to 0.95

              precision    recall  f1-score   support

         1.0       0.97      0.96      0.97       496
         2.0       0.98      0.96      0.97       471
         3.0       0.96      0.98      0.97       420
         4.0       0.87      0.91      0.89       491
         5.0       0.93      0.87      0.90       532
         6.0       0.99      1.00      0.99       537

    accuracy                           0.95      2947
   macro avg       0.95      0.95      0.95      2947
weighted avg       0.95      0.95      0.95      2947



In [97]:
from sklearn import svm

svm = svm.SVC()

In [98]:
svm.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [99]:
svm.score(X_train, y_train) # adding corr 0.9838 to 0.9857

0.9857181719260065

In [100]:
y_pred_svm = svm.predict(X_test)

In [102]:
print(classification_report(y_test, y_pred_svm)) # adding corr 0.96 to 0.96 

              precision    recall  f1-score   support

         1.0       0.99      0.98      0.99       496
         2.0       0.99      0.99      0.99       471
         3.0       0.98      0.98      0.98       420
         4.0       0.90      0.89      0.90       491
         5.0       0.91      0.90      0.91       532
         6.0       0.99      1.00      1.00       537

    accuracy                           0.96      2947
   macro avg       0.96      0.96      0.96      2947
weighted avg       0.96      0.96      0.96      2947



In [103]:
from sklearn.ensemble import RandomForestClassifier

In [104]:
rfc = RandomForestClassifier()

In [105]:
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [106]:
rfc.score(X_train, y_train) # 1.0 already without corr

1.0

In [107]:
y_pred_rfc = rfc.predict(X_test)

In [109]:
print(classification_report(y_test, y_pred_rfc)) # adding corr 0.92 to 0.94

              precision    recall  f1-score   support

         1.0       0.98      0.95      0.97       496
         2.0       0.89      0.97      0.93       471
         3.0       0.93      0.86      0.90       420
         4.0       0.86      0.97      0.91       491
         5.0       0.96      0.86      0.91       532
         6.0       1.00      1.00      1.00       537

    accuracy                           0.94      2947
   macro avg       0.94      0.93      0.93      2947
weighted avg       0.94      0.94      0.94      2947



In [110]:
# adding corr led to marginal improvements but nothing dramatic
# things look good so far
# code is still very repetitive and can be cleaned up with functions to load 3 sets of data and run in parallel
# will create one final file with just SVM analysis and run hyperparamter optimization and add a confusion matrix